In [20]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [21]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [22]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [23]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [24]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [25]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [26]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [27]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [28]:
use_case_directory_path = r"D:\Dissertation-GPT\dataset\SpecificationData\Book(Github)"
# save_path = r"D:\Dissertation-GPT\evaluate\ResultSet\6.24\shopee - best"

In [29]:
usecase_name_list = []
project_name = "book"

In [30]:
# for filename in os.listdir(use_case_directory_path):
#     usecase_name_list.append (filename.split(".txt")[0])
# print(usecase_name_list)

In [31]:
test_scenario_directory_path = r"D:\Dissertation-GPT\evaluate\ResultSet\6.25\book ts"
save_testcase_path = r"D:\Dissertation-GPT\evaluate\ResultSet\6.24\book - best\tc"
save_testcase_raw_path = r"D:\Dissertation-GPT\evaluate\ResultSet\6.24\book - best\tc raw"

In [32]:
usecase_name_list = []
for filename in os.listdir(test_scenario_directory_path):
    usecase_name_list.append (filename.split(".txt")[0])
print(usecase_name_list)

['Remove the author', 'Remove the book', 'Remove the genre']


In [33]:

for usecase in usecase_name_list:
  base_name = os.path.basename(usecase)
  usecase_name =usecase
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  testscenario_path = os.path.join (test_scenario_directory_path,f"{usecase_name}.txt" )
  testscenario_list = read_file_to_list(testscenario_path)
  print(testscenario_path)
  print(testscenario_list)
  for testscenario in testscenario_list:
    content = "Test scenarios:"+ testscenario+  "\nUse case:" + read_file_content(usecase_path)
    print(content)
    testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
    testcase_path = os.path.join (save_testcase_path,f"{usecase_name}-{testscenario}.txt" )
    testcase_raw_path = os.path.join (save_testcase_raw_path,f"{usecase_name}-{testscenario}.txt" )
    promptTestCase = [
    { "role": "system", "content": SYSTEM_PROMPT_8},
    { "role": "user", "content": content}
    ]
    gpt_response = ask(promptTestCase, client, model)
    json_data = json.loads(gpt_response)
    write_json_to_txt_raw(json_data, testcase_raw_path)

    promptSelfRef = [
        { "role": "system", "content": SELF_REF},
        { "role": "user", "content": "Test scenario: "+ testscenario + "\n" + gpt_response}
    ]
    final_response = ask(promptSelfRef, client, model)
    json_fin = json.loads(final_response)
    write_json_to_txt(json_fin, testcase_path)
  print("f{usecase_name} gen test case complete")

D:\Dissertation-GPT\evaluate\ResultSet\6.25\book ts\Remove the author.txt
['1. Successful Deletion Scenario', '2. Deletion Cancellation Scenario', '3. User Cancels Deletion', '4. User Navigates Away Using Menu Before Confirming Deletion']
Test scenarios:1. Successful Deletion Scenario
Use case:Remove the author

Main scenario:
User clicks the "Delete" button (to the right from the author he/she wants to delete) on the author list/details page;
Application displays a dialog page to confirm destroying the record for chosen author;
User confirms deletion by pressing "Confirm" button.

Cancel operation scenario:
User clicks the "Delete" button (to the right from the author he/she wants to delete) on the author list/details page;
Application displays a dialog page to confirm destroying the record for chosen author;
User presses “Cancel” button;
Data doesn’t delete in the database, then a list of authors records is displayed to the user;
If the user selects the menu item "Books”, "Genres”, "

In [34]:

# for usecase in usecase_name_list:
#   base_name = os.path.basename(usecase)
#   usecase_name =usecase
#   usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
#   testscenario_path = os.path.join (test_scenario_directory_path,f"{usecase_name}.txt" )
#   testscenario_list = read_file_to_list(testscenario_path)
#   print(testscenario_path)
#   print(testscenario_list)
#   for testscenario in testscenario_list:
#     content = "Test scenarios:"+ testscenario+  "\nUse case:" + read_file_content(usecase_path)
#     print(content)
#     testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','')
#     testcase_path = os.path.join (save_testcase_path,f"{usecase_name}-{testscenario}.txt" )
#     testcase_raw_path = os.path.join (save_testcase_raw_path,f"{usecase_name}-{testscenario}.txt" )
#     promptTestCase = [
#     { "role": "system", "content": SYSTEM_PROMPT_8},
#     { "role": "user", "content": content}
#     ]
#     gpt_response = ask(promptTestCase, client, model)
#     json_data = json.loads(gpt_response)
#     write_json_to_txt_raw(json_data, testcase_raw_path)

#     promptSelfRef = [
#         { "role": "system", "content": SELF_REF},
#         { "role": "user", "content": "Test scenario: "+ testscenario + "\n" + gpt_response}
#     ]
#     final_response = ask(promptSelfRef, client, model)
#     json_fin = json.loads(final_response)
#     write_json_to_txt(json_fin, testcase_path)
#   print("f{usecase_name} gen test case complete")